This Notebook will create a collaborative learing algorithm for recommending NFTs

Import NFT dataset. It can be downloaded from this URL : https://www.kaggle.com/datasets/hemil26/nft-collections-dataset?resource=download

Define package usage

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Get data from file

In [2]:
# Load the dataset
df = pd.read_csv('nft_sales.csv')

# Convert the collections column to a list of strings
collections = df['Collections'].tolist()
sales = df['Sales'].tolist()

Fit model

In [3]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the vectorizer on the collections column
tfidf_matrix = tfidf_vectorizer.fit_transform(sales)

Define users ownership as a dictionary with the name of collections followed by the number owned from that collection

In [4]:
# Get the user's current NFT ownership by collection
user_ownership = {'CryptoKitties': 8, 'Art Blocks': 2, 'NBA Top Shot': 1, 'Meebits': 1}

Run the model to get recommended nft collections

In [5]:
# Calculate the TF-IDF matrix for the user's collections
user_collection_descriptions = []
for collection in user_ownership.keys():
    description = df[df['Collections'] == collection]['Sales'].values[0]
    user_collection_descriptions.append(description)
user_tfidf_matrix = tfidf_vectorizer.transform(user_collection_descriptions)

# Calculate the cosine similarity between the user's collections and all other collections
cosine_similarities = cosine_similarity(user_tfidf_matrix, tfidf_matrix)

# Get the indices of the top 5 most similar collections
dic = {}
for i in range(len(cosine_similarities)):
    for j in range(len(cosine_similarities[i])):
        comp = cosine_similarities[i, j]
        if 0 < comp < 1:
            collection_name = str(df.iloc[j].at['Collections'])
            value = (list(user_ownership.values())[i] / sum(user_ownership.values())) * comp
            if collection_name in dic:
                dic[collection_name] += value
            else:
                dic[collection_name] = value

sorted_dic = dict(sorted(dic.items(), key=lambda x: x[1], reverse=True))
col_names = list(sorted_dic.keys())[:5]
print(col_names)

['ZombieClub Token', 'Boss Beauties', 'Autoglyphs', 'Treeverse', 'Crypto Unicorns']
